# Transate cloud data to beta 

This Code translate **lwc - liquid water content* data (from a txt file) and **cross section** (from scat file according to the effective radius) into **extinction** (beta)

===

## Reading the medium txt file:
in the txt file the first 6 row will look like:

```txt
#name=Stratocumulus water cloud
#original_cloud_data=@storm/data2/gmatheou/Factory/jplles/rev153/rico.20.smagorinsky/MISR01/An
#resampled_cloud_data_grid_size=20,20,40m
1024 1024 65 3
0.020 0.020     0.000     0.040     0.080     0.120     ...     2.520     2.560     2.600
230 977 64 2.6620E-01 2.2502E+01
```

The third row specifies the grid resolution, in this example  $(dx, dy, dz) = (20, 20, 40) [m]$.

The forth row specifies the grid size, in the example medium.shape will result  $(1024, 1024, 65)$. 

And the fifth row specifies the spacing the size of x,y voxels and the z grid, meaning  $dx,\ dy,\ z_0,\ z_1,\ z_2,\ \cdots,\  z_{N_z}$

From the sixth row till the end of the file, each row specifies the data in each voxel, for example:

|   X index   |   Y index   |   Z index   |   lwc $[\frac{g}{m^3}]$   |  Effective radius $[\mu m]$  |
| :---------: | :---------: | :---------: | :-------------: | :--------------------------: |
| 230         | 977         |  64         | 2.6620E-01      |  2.2502E+01                  |


And the relation between the extinction to lwc and effective radius is,
$$extinction = lwc\cdot cross\_section(r_{eff}) = Q \frac{\pi r_{eff}^2}{\frac{4}{3}\pi {r_{eff}}^3\rho}$$

since $\rho\simeq1 [\frac{g}{cm^3}]$, we can calaulate the foloowing,
$$extinction = Q \frac{\pi r_{eff}^2}{\frac{4}{3}\pi {r_{eff}}^3}$$

---

## Reading the scat file:
in this file the first 11 row will look like:

```txt
! Mie scattering table vs. effective radius (LWC=1 g/m^3)
    0.672    0.672  wavelength range (micron)
 1.000  W   particle density (g/cm^3) and type (Water, Ice, Aerosol)
  0.133100E+01 -0.212827E-07  particle index of refraction
gamma size distribution shape parameter
  50    5.000   25.000  number, starting, ending effective radius
  50    0.010    0.200  number, starting, ending effective variance
  50   97.000    2.000  number, starting, ending shape parameter alpha
  1222  maximum number of legendre coefficients allowed
   5.0000   0.0100  0.32360E+03 0.999998    144  Reff	 Veff  Ext  Alb  Nleg
      1.00000    2.54237    3.87643    4.55551    5.22360    5.86313    6.436 ...
```

The second row specifies the wavelength it was calculated to, in this example it's $0.672 [\mu m]$.

The next row specifies the partical density in $[\frac{g}{cm^3}]$ and the types of particals. The distribution is by gamma size distribution.

From the tenth row till the end of the file specifies folowing parameters and the legendre coefficiens pf the phase function:

| Effective radius $[\mu m]$ | Effective variance | Calculated cross section |    albedo | Number of legendre coefficients |
| :------------------------: | :----------------: | :----------------------: | :-------: | :-----------------------------: |
| 5.0000                     | 0.0100             |  0.32360E+03             | 0.999998  |  144                            |

**HG phase function relation to Mie phase function**
$g = $  taking the second legendre coefficien of Mie phase function by 3

## Assumptions

* Effective varince $= 0.1031$
* Albedo $= 1$
* Wavelete $672\ [nm]$
* Cross - section from mie function
* Size distribution gamma
* Lwc and effective radius data is from LES


## initialization

In [1]:
import numpy as np
import scipy.io as sio

## Parsering files

 ### Mie .scat file

In [2]:
# Open a file
f = open("Water_672nm.scat", "r+")

scat_lines = f.readlines()

# Close opend file
f.close()

wavelength = float(scat_lines[1].split()[0])

# effective variance is chosen to be 0.1 - selecting the closest greater equal to 0.1
var_eff_vec = np.array(scat_lines[6].split()[:3], dtype=float)
var_eff_vec = np.linspace(var_eff_vec[1], var_eff_vec[2], int(var_eff_vec[0]))
var_eff_par = round(var_eff_vec[ var_eff_vec >= 0.1 ][0], 4)

num_var_eff = len(var_eff_vec)
num_r_eff   = int(scat_lines[5].split()[0])

g              = np.zeros(num_r_eff)
r_eff_scat     = np.zeros(num_r_eff)
var_eff_scat   = np.zeros(num_r_eff)
cross_sec_scat = np.zeros(num_r_eff)

ind1 = -1
ind2 = -1

for line_ind, line in enumerate(scat_lines[9:]):
    line_arr = np.array(line.split())
    if (line_arr[-1] == 'Nleg'):
        if (float(line.split()[1]) == var_eff_par ):
            ind1                += 1
            r_eff_scat[ind1]     = float(line.split()[0])
            var_eff_scat[ind1]   = float(line.split()[1])
            cross_sec_scat[ind1] = float(line.split()[2])
        
    elif (float(line_arr[0]) == 1.) and (ind2 < ind1):
        ind2   += 1
        g[ind2] = float(line.split()[1]) / 3.


### LWC text file

In [3]:
# Open a file
f = open("rico20sn1024x1024x65.txt", "r+")

lwc_lines = f.readlines()

# Close opend file
f.close()

nx = int(lwc_lines[3].split()[0])
ny = int(lwc_lines[3].split()[1])
nz = int(lwc_lines[3].split()[2])

x_min = 0.0
y_min = 0.0
z_min = float(lwc_lines[4].split()[2])
z_max = float(lwc_lines[4].split()[-1])

cloud_field = np.zeros((nx, ny, nz))

for line in lwc_lines[5:]:
    [ indx, indy, indz ] = np.array(line.split()[:3], dtype=int) - 1
    [ lwc_s, r_eff_s ]   = np.array(line.split()[3:], dtype=float)
    if r_eff_s < r_eff_scat.min():
        cloud_field[ indx, indy, indz ] = 0.
    else:
        nearst_reff_ind = np.argmin(abs(r_eff_scat - r_eff_s))
        cloud_field[ indx, indy, indz ] = lwc_s * cross_sec_scat[nearst_reff_ind]


In [4]:
filename_out = 'rico20sn1024x1024x65_water_672nm.mat'

sio.savemat(filename_out, {'beta' : cloud_field})
print( 'extinction calculation is saved to ' + filename_out )

extinction calculation is saved to rico20sn1024x1024x65_water_672nm.mat
